In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
def get_columbiasportswear_details():
    links=set()
    for i in range(1,3):
        response = requests.get(f"https://columbiasportswear.co.in/hiking-activity-footwear&sort=sort_order&order=desc&page={i}")
        soup = BeautifulSoup(response.text, 'html.parser')
        
        for i in soup.find_all('a',attrs={'class':"single-product"}):
            links.add(i['href'])
    
    def get_shoe_name(soup):
        return soup.find("h1",attrs={'class':"product-title"}).text.strip()

    def get_category(soup):
        category = "Women" if "women" in get_shoe_name(soup).lower() else "Men"
        gender_label = soup.find('td', class_='specification-lable', string='Gender')

        if gender_label:
            gender_value = gender_label.find_next_sibling('td')
            if gender_value:
                category = gender_value.get_text(strip=True)
        return category

    def get_style_or_product_code(soup):
        return soup.find("div",attrs={'class':"col-sm-5"})('p')[0].text.strip()

    def get_reviews(soup):
        return soup.find("div",attrs={'class':"rating"}).find('a').text.strip().replace(' reviews', '')

    def get_sizes(soup):
        return [i.text.strip() for i in soup.select('.onhover')]
    
    def get_colors(soup):
        color_elements = soup.select('.grop-product-product a[data-color]')
        return [element['data-color'] for element in color_elements]

    def get_star_count(soup):
        return soup.select_one('.actual-star').text.strip().replace('of 5', '')

    def get_price(soup):
        return soup.select_one('h2').text.strip().replace(',', '')
    
    details = []
    for index, link in enumerate(links):
        print(f"Trying {index} - {link}")
        link_detail = {"link": link, "brand": "columbiasportswear"}
        response = requests.get(link)
        soup = BeautifulSoup(response.text, 'html.parser')
        link_detail["shoe_name"] =  get_shoe_name(soup)
        link_detail["category"] = get_category(soup)
        link_detail["style_or_product_code"] = get_style_or_product_code(soup)
        link_detail["reviews"] = get_reviews(soup)
        available_sizes = get_sizes(soup)
        link_detail["number_of_sizes"] = len(available_sizes)
        link_detail["size"] = ",".join(available_sizes)
        colors_available = get_colors(soup)
        link_detail["number_of_colors"] = len(colors_available)
        link_detail["color_1"], link_detail["color_2"], link_detail["color_3"], link_detail["color_4"], link_detail["color_5"] = (colors_available + [None] * (5 - len(colors_available)))[:5]
        link_detail["stars"] = get_star_count(soup)
        link_detail["price"] = get_price(soup)
        link_detail["comfort"] = None
        link_detail["quantification"] = None
        details.append(link_detail)
    
    return details

columbiasportswear_details = get_columbiasportswear_details()
print(columbiasportswear_details)


Trying 0 - https://columbiasportswear.co.in/hiking-activity-footwear/trailstorm-waterproof-bl0156-193
Trying 1 - https://columbiasportswear.co.in/hiking-activity-footwear/columbia-women-black-facet-75-outdry-bl8538-010
Trying 2 - https://columbiasportswear.co.in/hiking-activity-footwear/trailstorm-beyond-bl0354-439
Trying 3 - https://columbiasportswear.co.in/hiking-activity-footwear/columbia-men-black-strata-trail-mid-wp-ym8651-010
Trying 4 - https://columbiasportswear.co.in/hiking-activity-footwear/columbia-men-brown-re-peak-mid-bm6940-247
Trying 5 - https://columbiasportswear.co.in/hiking-activity-footwear/hatana-max-outdry-bm0659-013
Trying 6 - https://columbiasportswear.co.in/hiking-activity-footwear/peakfreak-ii-mid-outdry-bl7573-089
Trying 7 - https://columbiasportswear.co.in/hiking-activity-footwear/columbia-men-brown-crestwood-waterproof-bm5372-255
Trying 8 - https://columbiasportswear.co.in/hiking-activity-footwear/trailstorm-waterproof-bm0156-010
Trying 9 - https://columbiasp

In [9]:
df1_columns = ["shoe_name", "category", "number_of_colors", "price"]
df2_columns = ["number_of_sizes", "color_1","color_2","color_3","color_4","color_5","style_or_product_code"]
df3_columns = ["reviews", "size", "comfort", "quantification", "stars"]
columbiasportswear_df1 = pd.DataFrame({key: d.get(key) for key in df1_columns} for d in columbiasportswear_details)
columbiasportswear_df2 = pd.DataFrame({key: d.get(key) for key in df2_columns} for d in columbiasportswear_details)
columbiasportswear_df3 = pd.DataFrame({key: d.get(key) for key in df3_columns} for d in columbiasportswear_details)

print(columbiasportswear_df1)
print("\n")

print(columbiasportswear_df2)
print("\n")
print(columbiasportswear_df3)
print("\n")

                                            shoe_name category  \
0   Columbia Women White TRAILSTORM Water Resistan...    Women   
1   Columbia Women Black FACET 75 OUTDRY (Complete...    Women   
2               Columbia Women Blue TRAILSTORM BEYOND    Women   
3              Columbia Men Black STRATA TRAIL MID WP      Men   
4                      Columbia Men Brown RE-PEAK MID      Men   
5   Columbia Men Black HATANA MAX OUTDRY (Complete...      Men   
6   Columbia Women Grey PEAKFREAK II MID OUTDRY (C...    Women   
7   Columbia Men Brown Crestwood Water Resistant S...      Men   
8   Columbia Men Black TRAILSTORM Water Resistant ...      Men   
9   Columbia Women Black MINX SLIP IV Water Resist...    Women   
10               Columbia Men Black TRAILSTORM BEYOND      Men   
11  Columbia Men Brown Peakfreak II Outdry (Comple...      Men   
12  Columbia Women Grey TRAILSTORM Water Resistant...    Women   
13  Columbia Women Red PEAKFREAK X2 OUTDRY (Comple...    Women   
14  Columb

In [10]:
all_df = pd.DataFrame(columbiasportswear_details)
all_df.to_csv("columbia_all.csv")

columbiasportswear_df1.to_csv("columbia_1.csv")
columbiasportswear_df2.to_csv("columbia_2.csv")
columbiasportswear_df3.to_csv("columbia_3.csv")
